In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = '/home/big_muscle/data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (500000, 28, 28) (500000,)
Validation set (20000, 28, 28) (20000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    """
    这个函数需要把三维的图片变成一个二维矩阵，同时把标签也排好。
    """
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32) 
    # This -1 means the length of the "dataset" array.
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    # 原始数据的label是 0~9 的是个数字，50000*1的矩阵， 现在变为50000*10的矩阵。
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (500000, 784) (500000, 10)
Validation set (20000, 784) (20000, 10)
Test set (10000, 784) (10000, 10)


第一步
把输入数据，变量，计算方法，描述出来
然后在显卡上跑上述的数据，接着显示出来。

In [4]:
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  # bias初始化是0.
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [5]:
num_steps = 10000

def accuracy(predictions, labels):
    # 所有预测正确的个数除以所有的预测，所得到的比例作为准确率。
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 16.563282
Training accuracy: 14.9%
Validation accuracy: 17.6%
Loss at step 100: 2.355880
Training accuracy: 72.3%
Validation accuracy: 70.5%
Loss at step 200: 1.910303
Training accuracy: 74.9%
Validation accuracy: 72.7%
Loss at step 300: 1.658671
Training accuracy: 76.1%
Validation accuracy: 73.6%
Loss at step 400: 1.487458
Training accuracy: 76.8%
Validation accuracy: 74.1%
Loss at step 500: 1.359588
Training accuracy: 77.7%
Validation accuracy: 74.5%
Loss at step 600: 1.259303
Training accuracy: 78.4%
Validation accuracy: 74.7%
Loss at step 700: 1.178150
Training accuracy: 79.0%
Validation accuracy: 74.9%
Loss at step 800: 1.110823
Training accuracy: 79.5%
Validation accuracy: 75.0%
Loss at step 900: 1.053852
Training accuracy: 79.9%
Validation accuracy: 75.0%
Loss at step 1000: 1.004858
Training accuracy: 80.2%
Validation accuracy: 75.1%
Loss at step 1100: 0.962156
Training accuracy: 80.5%
Validation accuracy: 75.2%
Loss at step 1200: 0.924535
Training ac

上面的优化方法使用的是gradient descent， 下面使用stochastic descent， 需要使用一个preholder，用来存储临时变量，类似于一个循环器。

In [6]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  # train_dataset 并没有导入到placeholder？
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size) # 这一步的offset的作用是什么？ 
    # 貌似想使用随机化。
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :] # 以batchsize作为步伐来遍历整个训练集
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.462795
Minibatch accuracy: 12.5%
Validation accuracy: 14.7%
Minibatch loss at step 500: 2.129686
Minibatch accuracy: 75.8%
Validation accuracy: 75.5%
Minibatch loss at step 1000: 1.089851
Minibatch accuracy: 78.1%
Validation accuracy: 76.9%
Minibatch loss at step 1500: 1.253541
Minibatch accuracy: 77.3%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 0.886859
Minibatch accuracy: 78.1%
Validation accuracy: 77.3%
Minibatch loss at step 2500: 0.829983
Minibatch accuracy: 78.1%
Validation accuracy: 78.4%
Minibatch loss at step 3000: 1.205320
Minibatch accuracy: 73.4%
Validation accuracy: 78.4%
Test accuracy: 85.6%


# 一步步解析

In [8]:
train_dataset.shape

(500000, 784)

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units nn.relu() and 1024 hidden nodes. This model should improve your validation / test accuracy.

In [10]:
a = tf.constant([1.0, 2.0], name='a')

In [11]:
b = tf.constant([3.0, 4.0], name='b')

In [12]:
result = a + b

In [13]:
result

<tf.Tensor 'add:0' shape=(2,) dtype=float32>

In [14]:
a+b

<tf.Tensor 'add_1:0' shape=(2,) dtype=float32>

In [17]:
with tf.Session() as sess:
    sess.run(result)

In [19]:
sess = tf.Session()

In [20]:
sess.close()

In [22]:
sess.close()

In [23]:
sess.run(result)

array([ 4.,  6.], dtype=float32)